In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

###### Shift Detector Function

In [2]:
def create_sift_detector_image_new_task3(img, nf, nOL, cT, eT, si):
    sift_create = cv2.SIFT_create(nfeatures=nf, nOctaveLayers=nOL, contrastThreshold=cT, edgeThreshold=eT, sigma=si)
    keypt, desc = sift_create.detectAndCompute(img, None)
    
    number_of_keypts = len(keypt)##'keypt' is a list
    
    reqd_keypts = int(0.1 * len(keypt))## reguired number of keypoints = 10/100 * keypoints in original image
    
    sift_create.setNFeatures(reqd_keypts)###setting nfeatures to required number of keypoints
    
    
    new_keypts, desc = sift_create.detectAndCompute(img, None) ##getting the new keypoints and descriptive features
    
    
    return new_keypts, desc

###### Function for Making Negative Input Image

In [3]:
def negative_image(img):
    img_arr = np.zeros(img.shape, dtype=img.dtype)
    
    for i in range(img_arr.shape[0]):
        for j in range(img_arr.shape[1]):
            for k in range(img_arr.shape[2]):
                img_arr[i][j][k] = 255 - img[i][j][k]##subtracting 255 from pixel value in each channel
                
    return img_arr            

###### Function for Making Brute Force Matches with the Original and Negative Image

In [4]:
def Brute_Force_Matches(desc_neg, desc_img):
    bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=False)

    matches = bf.match(desc_img,desc_neg)
    
    ##sorting the matching points according to nearest distance
    matches = sorted(matches, key = lambda x:x.distance)
    
    return matches

###### Function to return Image with bounding box

In [5]:
def img_with_bbox(matches, keypts, org_img):
    first_match_pts = []

    for i in matches:
        
        ##getting the matching coordinates
        matching_coords = keypts[i.queryIdx].pt
        first_match_pts.append(matching_coords)
        
    ##converting the list to array for storing the keypoints
    keypoints_first = np.float32(first_match_pts).reshape(-1, 1, 2)
    
    ##getting the x and y coordinates
    x_coordinates = keypoints_first[:, 0, 0]
    y_coordinates = keypoints_first[:, 0, 1]
    
    
    ##getting the bottomost right x and topmost left x-cordinates
    max_x = np.max(x_coordinates)
    min_x = np.min(x_coordinates)
    
    ####getting the bottomost right y and topmost left y-cordinates
    max_y = np.max(y_coordinates)
    min_y = np.min(y_coordinates)
    
   
    cr_x_min = int(min_x)
    cr_y_min = int(min_y)
    cr_x_max = int(max_x)
    cr_y_max = int(max_y)
    
    ###calculating the width and height of the bounding box
    width_bbox = cr_x_max - cr_x_min
    
    height_bbox = cr_y_max - cr_y_min

    return cr_x_min, cr_y_min, (width_bbox + cr_x_min) , (height_bbox + cr_y_min)

##### Custom Algorithm when length of matching points < 55

In [6]:
def Contour_Detect_Algo_OpenCV(img):
    
    ##Saving the Copy of the Image
    img_copy = img.copy()
    
    
    ##Step 1: Convert the Image to Grayscale
    img_gray = cv2.cvtColor(img_copy, cv2.COLOR_BGR2GRAY)
    
    ##Step 2: Reversing the Intensity of the Image
    reverse_gray = cv2.bitwise_not(img_gray)
    
    #Step 3: Binary Thresholded Image formation with thresh=180
    _, img_binary = cv2.threshold(reverse_gray, 180, 255, cv2.THRESH_BINARY)
    
    
    #Step 4: finding all the contours from the thresholded image
    contours, hierarchy = cv2.findContours(img_binary, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
    ##checking if the length of contours or number of contours > 0
    if len(contours) > 0:
        
        ###evaluating the largest area of the contour
        biggest_contour = max(contours, key = cv2.contourArea)
        
        ##getting the coordinates of the bounding box
        x, y, w, h = cv2.boundingRect(biggest_contour)
            
        x_min = x
        y_min = y
    
        x_max = x + w
        y_max = y + h
        
    else:
        x_min = 0   ##if still number of contours detected in image is 0, returning original input image
        y_min = 0
        x_max = img_copy.shape[0]
        y_max = img_copy.shape[1]
    
    return x_min, y_min, x_max , y_max

##### Main Function For the Proposed Detection Method

In [8]:
def detection_T3(img):
    
    img_copy = img.copy()
    ##Step 1: Conversion to negative Image
    neg_img = negative_image(img)
    #plt.imshow(neg_img)
    
    ## Step 2: getting shift descriptor points from original image
    keypts_img, desc_img = create_sift_detector_image_new_task3(img, 0, 4, 0.04, 50, 1.67)
    
    
    ##Step 3: getting shift descriptor points from negative image
    keypts_neg_img, desc_neg = create_sift_detector_image_new_task3(neg_img, 0, 4, 0.04, 50, 1.67)
    
    ## Step4: finding matching keypoints between the negative and original image
    matching_keypts = Brute_Force_Matches(desc_neg, desc_img)
    
   
    
    ###Step 5: Bounding the image with bounding box
    if len(matching_keypts) > 55:
        x_min, y_min, x_max, y_max = img_with_bbox(matching_keypts, keypts_img, img)
        
    else:
        
        x_min, y_min, x_max, y_max = Contour_Detect_Algo_OpenCV(img_copy)
    
    return x_min, y_min, x_max, y_max 
       

##### Loading Dataset Function for Training Directory

In [9]:
import os
def loading_of_dataset(directory):
    imgs_dir = os.listdir(directory)
    
    
    count = 0
    for label in imgs_dir:
        img_dir = os.path.join(directory, label)##taking the full image directory
        image = cv2.imread(img_dir)
        img_rgb = image[:,:,::-1]
        min_x, min_y, max_x, max_y = detection_T3(image)
        
        cropped_image = image[min_y:max_y, min_x:max_x]
        
        cv2.imwrite('Final_Train_Set/{}.jpg'.format(count), cropped_image)
        count = count + 1
        
        
         

###### Loading the Trainset Directory Images

In [10]:
train_directory = 'Penguins vs Turtles/archive/train/train'
loading_of_dataset(train_directory)

##### Loading Dataset to Validation Directory

In [11]:
import os
def loading_of_dataset(directory):
    imgs_dir = os.listdir(directory)
    
    predicted_coordinates = []
    count = 0
    for label in imgs_dir:
        img_dir = os.path.join(directory, label)##taking the full image directory
        image = cv2.imread(img_dir)
        img_rgb = image[:,:,::-1]
    
        
        
        min_x, min_y, max_x, max_y = detection_T3(image)
        
        
        predicted_coordinates.append([min_x, min_y, max_x, max_y])
        
       
        cropped_image = image[min_y:max_y, min_x:max_x]
    
        
        cv2.imwrite('Last_Valid_Set/{}.jpg'.format(count),  cropped_image)
        count = count + 1
        
    return predicted_coordinates    
         

In [12]:
validation_directory = 'Penguins vs Turtles/archive/valid/valid'
list_of_coords = loading_of_dataset(validation_directory)

In [13]:
list_of_coords

[[214, 107, 628, 620],
 [16, 113, 621, 526],
 [54, 108, 603, 533],
 [137, 117, 564, 531],
 [89, 165, 540, 633],
 [10, 30, 632, 602],
 [64, 62, 579, 474],
 [122, 16, 529, 512],
 [19, 24, 623, 538],
 [455, 401, 502, 470],
 [9, 78, 632, 559],
 [49, 0, 592, 295],
 [158, 31, 633, 355],
 [82, 15, 556, 622],
 [92, 125, 546, 634],
 [364, 296, 420, 404],
 [14, 174, 597, 532],
 [34, 24, 606, 589],
 [14, 80, 625, 579],
 [100, 210, 611, 556],
 [13, 135, 620, 532],
 [10, 2, 634, 593],
 [0, 259, 411, 533],
 [164, 260, 508, 534],
 [9, 76, 376, 559],
 [36, 210, 640, 411],
 [10, 108, 604, 522],
 [109, 172, 591, 531],
 [26, 289, 464, 446],
 [51, 204, 623, 516],
 [7, 55, 612, 593],
 [207, 323, 351, 411],
 [118, 188, 465, 486],
 [7, 266, 633, 559],
 [0, 430, 99, 486],
 [33, 82, 610, 634],
 [6, 79, 630, 560],
 [2, 198, 620, 532],
 [3, 102, 636, 544],
 [0, 108, 640, 532],
 [47, 98, 571, 636],
 [68, 240, 605, 530],
 [406, 255, 468, 312],
 [6, 80, 610, 565],
 [3, 108, 624, 532],
 [109, 232, 530, 614],
 [209, 

In [14]:
len(list_of_coords)

72

In [15]:
import json

In [16]:
valid_annotations_dir = 'valid_annotations'

In [17]:
with open(valid_annotations_dir, 'r') as f:
    valid_annotations_given = json.load(f)

In [18]:
valid_annotations_given[0]['bbox'][2]

298

In [19]:
actual_coordinates = []

for i in range(len(list_of_coords)):
    ##getting x_max = width + x_min
    x_min, y_min, width, height = valid_annotations_given[i]['bbox']
    actual_coordinates.append([x_min, y_min, x_min + width, y_min + height])

In [20]:
actual_coordinates

[[227, 93, 525, 618],
 [211, 198, 639, 457],
 [0, 243, 512, 510],
 [128, 178, 585, 532],
 [168, 131, 464, 600],
 [6, 121, 451, 592],
 [63, 161, 501, 483],
 [188, 252, 442, 528],
 [375, 237, 564, 552],
 [364, 223, 505, 469],
 [155, 173, 640, 533],
 [133, 271, 484, 387],
 [141, 39, 640, 421],
 [126, 414, 519, 565],
 [178, 119, 464, 602],
 [363, 187, 618, 406],
 [103, 167, 599, 433],
 [60, 167, 546, 640],
 [164, 284, 320, 357],
 [114, 205, 625, 428],
 [137, 111, 533, 516],
 [159, 321, 442, 548],
 [0, 228, 414, 436],
 [156, 248, 514, 533],
 [43, 138, 301, 526],
 [35, 209, 639, 412],
 [4, 169, 490, 519],
 [252, 168, 640, 386],
 [223, 256, 456, 372],
 [193, 231, 552, 454],
 [61, 127, 514, 580],
 [171, 297, 464, 440],
 [76, 179, 482, 534],
 [75, 306, 423, 450],
 [231, 333, 346, 396],
 [155, 161, 489, 486],
 [56, 186, 599, 460],
 [258, 294, 352, 468],
 [193, 152, 335, 515],
 [168, 180, 640, 414],
 [94, 251, 500, 573],
 [232, 230, 585, 405],
 [316, 252, 520, 509],
 [99, 81, 582, 561],
 [252, 21

In [21]:
len(actual_coordinates)

72

##### Testing Detection Model Coordinates

In [22]:
import math

In [23]:
def estimate_centre_pos(coords_list):
    
    ##getting the following coordinates
    x_min, y_min, x_max, y_max = coords_list
    
    ##estimating center pos of x-coordinate
    x_center = (x_min + x_max)/2 
    
    ##estimating center pos of y-coordinate
    y_center = (y_min + y_max)/2 
    
    
    return x_center, y_center   

In [24]:
def Euclid_Dist_coords(pred_list, actual_list):
    
    ##getting centre positions of predicted bounding box
    center_x_pred, center_y_pred = estimate_centre_pos(pred_list)
    
    ##getting center positions of actual bounding box
    center_x_actual, center_y_actual = estimate_centre_pos(actual_list)
    
    ##calculating the euclidean distance
    centre_distance = math.sqrt((center_x_pred - center_x_actual)**2 + (center_y_pred - center_y_actual)**2)
    
    return centre_distance  

In [25]:
len(list_of_coords)

72

In [26]:
len(actual_coordinates)

72

In [27]:
measures_dist = []
for i, j in zip(list_of_coords, actual_coordinates):
    dist = Euclid_Dist_coords(i, j)
    measures_dist.append(dist)

In [28]:
measures_dist

[45.70557952810576,
 106.80004681646913,
 91.6092244263644,
 31.575306807693888,
 33.53356527421443,
 100.97772031492887,
 66.90478308760892,
 126.43674307731911,
 186.90773124726542,
 99.73088789337032,
 84.37564814565872,
 181.89626164382818,
 37.33630940518894,
 171.0358149628317,
 19.1049731745428,
 112.09148049695838,
 69.85162847063768,
 98.47842403288143,
 78.02083055184686,
 67.9577074363166,
 27.244265451650556,
 138.67678248358663,
 64.01757571167468,
 6.576473218982953,
 25.10975905897944,
 1.0,
 66.64082832618455,
 121.51645979043333,
 108.59327787667154,
 39.5790348543266,
 36.800135869314396,
 38.52920969861697,
 23.16246964380094,
 78.93826701923471,
 256.63836424042296,
 34.50362299817223,
 10.124228365658293,
 17.08800749063506,
 56.48451115128819,
 87.09190547921202,
 46.57252408878007,
 98.6927048975759,
 98.84331034521254,
 32.53459696999488,
 77.26901060580497,
 118.00105931727902,
 38.08214804865923,
 184.56570645707723,
 111.5100892296298,
 97.62812094883319,
 58

###### Calculation of mean and standard deviation between centers of predicted and actual bounding box

In [29]:
mean_dist_btwn_centres = sum(measures_dist)/len(measures_dist)
print("Mean Distance between Center of Prdicted and Actual is: ", mean_dist_btwn_centres)

Mean Distance between Center of Prdicted and Actual is:  77.55542809528269


In [30]:
def calculate_Standard_dev(data, mean):
    
    #mean = calculate_mean(data)
    
    sum_squared_diff = 0
    for x in data:
        sq_diff = (x - mean) ** 2
        sum_squared_diff = sum_squared_diff + sq_diff
        
        
    variance = sum_squared_diff / len(data)
    
    standard_dev = math.sqrt(variance)

    return  standard_dev

In [31]:
standard_dev_btwn_centers = calculate_Standard_dev(measures_dist, mean_dist_btwn_centres)
print("Standard Deviation between Center of Prdicted and Actual is: ", standard_dev_btwn_centers)

Standard Deviation between Center of Prdicted and Actual is:  56.19125960761847


###### Testing Detection by IOU 

In [32]:
def estimate_intersect_area(pred_cords, actual_cords):
    
    x_min_pred, y_min_pred, x_max_pred, y_max_pred = pred_cords
    
    x_min_actual, y_min_actual, x_max_actual, y_max_actual = actual_cords
    
    ##common area in x-axis
    commomn_x_area = max(0, (min(x_max_pred, x_max_actual) - max(x_min_pred,  x_min_actual)))
    
    ##common-area in y-axis
    commomn_y_area = max(0, (min(y_max_pred, y_max_actual) - max(y_min_pred,  y_min_actual)))
    
    ##getting the common area between two images
    intersection_area = commomn_x_area * commomn_y_area

    
    return intersection_area

In [33]:
def estimate_union_area(pred_cords, actual_cords):
    
    x_min_pred, y_min_pred, x_max_pred, y_max_pred = pred_cords
    
    x_min_actual, y_min_actual, x_max_actual, y_max_actual = actual_cords
    
    
    area_pred =  (x_max_pred -  x_min_pred) * (y_max_pred - y_min_pred)
    
    area_actual = (x_max_actual -  x_min_actual) * (y_max_actual - y_min_actual)
    
    area_estimated_union = area_pred + area_actual - estimate_intersect_area(pred_cords, actual_cords)
    
    return area_estimated_union

In [34]:
def estimate_iou_calculate(pred_bbox, actual_bbox):
    
    area_intersected = estimate_intersect_area(pred_bbox, actual_bbox)
    area_union = estimate_union_area(pred_bbox, actual_bbox)
    estimate_iou = area_intersected / area_union
    return estimate_iou

In [35]:
measures_iou = []
for m, n in zip(list_of_coords, actual_coordinates):
    iou_img = estimate_iou_calculate(m, n)
    measures_iou .append(iou_img)

In [36]:
measures_iou

[0.7031871958033562,
 0.4172052473804351,
 0.4936002228115426,
 0.802507653400772,
 0.5822766492411772,
 0.5807368844850532,
 0.6319726476771317,
 0.3206821536788128,
 0.1816947831697019,
 0.09201623815967523,
 0.5675887394519126,
 0.043766268177496535,
 0.7720478556512257,
 0.2062540404145726,
 0.5860001202738808,
 0.10829975825946817,
 0.6015213630209116,
 0.5894024128794193,
 0.03735129834136358,
 0.593959860316973,
 0.60234027858178,
 0.17419681981864724,
 0.5774533850879908,
 0.9173422939418212,
 0.5647265896051585,
 0.9868906386131762,
 0.6773756531836655,
 0.3919795111197562,
 0.2529461774900268,
 0.448589071185225,
 0.6304617653384128,
 0.30244158571803625,
 0.7174495247346145,
 0.2732120075456062,
 0.0,
 0.3408120463165298,
 0.49570206300975533,
 0.0792395790942387,
 0.18423366430057259,
 0.4070165094339623,
 0.4637333636028264,
 0.3657227175687555,
 0.06740672922865644,
 0.791424865160101,
 0.1745814723665421,
 0.29279391375448827,
 0.6912995647390826,
 0.10102087460003048,
 

In [37]:
mean_iou = sum(measures_iou) / len(measures_iou)
mean_iou

0.419686318262846

In [38]:
print("Mean IOU between Center of Prdicted and Actual is: ", mean_iou)

Mean IOU between Center of Prdicted and Actual is:  0.419686318262846


In [39]:
standard_dev_btwn_centers_iou = calculate_Standard_dev(measures_iou, mean_iou)
print("Standard Deviation between Center of Prdicted and Actual is: ", standard_dev_btwn_centers_iou)

Standard Deviation between Center of Prdicted and Actual is:  0.2650276272925348
